In [3]:
import time
import copy
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

In [4]:
Z = np.load(open("./tsne_data_reducted_normalised.npy", "rb"))
Z = torch.tensor(Z, requires_grad=True, dtype=float)
Z.shape

torch.Size([70000, 3])

# develop VisualImitation

In [28]:
def myact(x):
    # return nn.Sigmoid()(x)
    return nn.Tanh()(x*100000)

# a = torch.arange(start=-1,end=1,step=0.0001,dtype=float)
# b = myact(a)

# plt.plot(a,b)

In [24]:
def get_m(zi, pts=0):
    a0 = torch.ones((1000,1000), requires_grad=True)
    a = a0 * zi[0] * 1000
    b0 = torch.ones((1000,1000), requires_grad=True)
    b = b0 * zi[1] * 1000
    if pts==0:
        gridx = torch.arange(1,1001)
        gridy = torch.arange(1,1001)
    elif pts==1:
        gridx = torch.arange(0,1000)
        gridy = torch.arange(1,1001)
    elif pts==2:
        gridx = torch.arange(2,1002)
        gridy = torch.arange(2,1002)
    elif pts==3:
        gridx = torch.arange(1,1001)
        gridy = torch.arange(0,1000)

    x_new = myact(a-gridx)
    y_new = myact(b-gridy.reshape(-1,1))
    m = myact(x_new*y_new)

    return x_new, y_new, m

In [25]:
f = torch.nn.Conv2d(1, 1, (3, 3), stride=1, padding=1, bias=False)
f.weight.data = torch.Tensor([[[[0, 1, 0],
                                [1, 0, 1],
                                [0, 1, 0]]]])


def myfunc(z, label=0, size=1000):
    # mask = torch.zeros((1000,1000), dtype=float)
    # i = torch.floor(z[0]*size).type(torch.LongTensor)
    # j = torch.floor(z[1]*size).type(torch.LongTensor)
    # print(i.dtype)
    # mask[i][j] = 1

    # _, _, mask1 = get_m(zi=z, pts=1)
    # _, _, mask3 = get_m(zi=z, pts=3)

    # mask = mask1 * mask3

    # masks = torch.sigmoid((f(mask.unsqueeze(0).unsqueeze(0)).squeeze()-3)*1000)

    # get mask_mat at point i
    _, _, mask1 = get_m(zi=z[:2], pts=1)
    _, _, mask3 = get_m(zi=z[:2], pts=3)

    # get the ultimate 0-1 mask
    mask = (mask1 * mask3 - 1) / 2 * (-1)
    mask = torch.transpose(mask, 0, 1)

    masks = torch.sigmoid((f(mask.unsqueeze(0).unsqueeze(0)).squeeze()-3)*1000)
    masks = masks * (z[2]+1)

    return masks

In [26]:
Z[2, :]

tensor([0.6714, 0.8661, 4.0000], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [29]:
_, _, mask1 = get_m(zi=Z[2, :2], pts=1)
_, _, mask3 = get_m(zi=Z[2, :2], pts=3)

mask = (mask1 * mask3 - 1) / 2 * (-1)
mask = torch.transpose(mask, 0, 1)

masks = torch.sigmoid((f(mask.unsqueeze(0).unsqueeze(0)).squeeze()-3)*1000)

In [39]:
# mask1[671, 866]

k = 3
(1/2-mask1 * mask3/2)[(866-k):(866+k+1),(671-k):(671+k+1)]

tensor([[0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 1., 1., 0., 1., 1., 1.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.]], grad_fn=<SliceBackward0>)

In [32]:
mask1, (mask1 * mask3 - 1) / 2 * (-1)

(tensor([[ 1.,  1.,  1.,  ..., -1., -1., -1.],
         [ 1.,  1.,  1.,  ..., -1., -1., -1.],
         [ 1.,  1.,  1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ...,  1.,  1.,  1.]], grad_fn=<TanhBackward0>),
 tensor([[-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         ...,
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.]], grad_fn=<MulBackward0>))

In [30]:
masks

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SigmoidBackward0>)

In [47]:
k = 3
mask[(671-k):(671+k+1),(866-k):(866+k+1)]

tensor([[-0., -0., -0., 1., -0., -0., -0.],
        [-0., -0., -0., 1., -0., -0., -0.],
        [-0., -0., -0., 1., -0., -0., -0.],
        [1., 1., 1., -0., 1., 1., 1.],
        [-0., -0., -0., 1., -0., -0., -0.],
        [-0., -0., -0., 1., -0., -0., -0.],
        [-0., -0., -0., 1., -0., -0., -0.]], grad_fn=<SliceBackward0>)

In [48]:
masks.shape, mask

(torch.Size([1000, 1000]),
 tensor([[-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         ...,
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.],
         [-0., -0., -0.,  ..., -0., -0., -0.]], grad_fn=<TransposeBackward0>))

In [49]:
torch.sum(masks)

tensor(1., grad_fn=<SumBackward0>)

In [50]:
masks = masks * (Z[2, 2]+1)

# transform mask matrix to onehot form
masks = masks.reshape(1000*1000,1)

res = F.one_hot(masks.long(), num_classes=11)
res = res.squeeze().reshape(1000,1000,-1)

In [51]:
torch.sum(res)

tensor(1000000)

In [52]:
res.shape

torch.Size([1000, 1000, 11])

In [53]:
# res[831][524]
res[671][866]

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [54]:
for i in range(len(res)):
    for j in range(len(res[i])):
        if res[i][j].argmax().item()!=0:
            print(i, j)

KeyboardInterrupt: 

In [ ]:
def myfunc1(masks, size=1000):
    # transform mask matrix to onehot form
    masks = masks.reshape(size*size,1)
    res = F.one_hot(masks.long(), num_classes=11)
    res = res.squeeze().reshape(size,size,-1)

    return res

In [ ]:
# z1024 = Z[:1024].detach()
z1024 = Z[:64].detach()
z1024.requires_grad_(True)
z1024

In [ ]:
masks = torch.vmap(myfunc)(z1024)
masks.shape

In [ ]:
masks1 = torch.sum(masks, dim=0)
masks1.shape

In [ ]:
from utils import draw_Ihat

draw_Ihat(I_hat=masks1.detach())

In [ ]:
res = myfunc1(masks1)
res.shape

In [ ]:
label = torch.ones((1000,1000,11))
label = label.type(torch.DoubleTensor)
label.requires_grad_(False)
label.shape

In [ ]:
I_hat = res * label
I_hat.shape

In [ ]:
I_hat1 = torch.sum(I_hat, dim=0)
I_hat1.shape

In [ ]:
gt = torch.randint(0,11,(1000,1000))
gt = gt.type(torch.DoubleTensor)
gt.requires_grad_(False)
gt.shape

In [ ]:
criterion = nn.L1Loss()

loss = criterion(gt, I_hat1)
loss.backward()

In [ ]:
z1024.grad

In [ ]:
torch.sum(z1024)

# convolution to detect centre point

In [23]:
mat = torch.zeros((11,11), dtype=float)

mat[5,:] = torch.ones((mat[5,:].shape))
mat[:, 5] = torch.ones((mat[:,5].shape))
mat[5,5] = 0
mat = mat.unsqueeze(0).unsqueeze(0)
mat, mat.shape

(tensor([[[[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]]], dtype=torch.float64),
 torch.Size([1, 1, 11, 11]))

In [24]:
c = torch.nn.Conv2d(1, 1, (3, 3), stride=1, bias=False)
c.weight.data = torch.Tensor([[[[0, 1, 0],
                                [1, 0, 1],
                                [0, 1, 0]]]])
c.weight.data, c.weight.data.shape

(tensor([[[[0., 1., 0.],
           [1., 0., 1.],
           [0., 1., 0.]]]]),
 torch.Size([1, 1, 3, 3]))

In [25]:
res = c(mat.float())
res

tensor([[[[0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [1., 1., 1., 2., 1., 2., 1., 1., 1.],
          [2., 2., 2., 1., 4., 1., 2., 2., 2.],
          [1., 1., 1., 2., 1., 2., 1., 1., 1.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.]]]],
       grad_fn=<ConvolutionBackward0>)

In [26]:
conv_weights = torch.Tensor([[[[0, 1, 0],
                            [1, 0, 1],
                            [0, 1, 0]]]])
res = F.conv2d(input=mat.float(), weight=conv_weights)                            
res

tensor([[[[0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [1., 1., 1., 2., 1., 2., 1., 1., 1.],
          [2., 2., 2., 1., 4., 1., 2., 2., 2.],
          [1., 1., 1., 2., 1., 2., 1., 1., 1.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.],
          [0., 0., 0., 1., 2., 1., 0., 0., 0.]]]])

In [32]:
torch.relu(res-3)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.]]]])

In [27]:
res1 = torch.sigmoid((res-3)*1000)
res1

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0.]]]])

In [28]:
res1.shape

torch.Size([1, 1, 9, 9])

# demo VisualImitation

In [5]:
from utils import *
from models import *

VImodule = VisualImitation(size=100, device=torch.device('cuda'))

In [6]:
# z = minmax(Z[128:256, :]).cuda()
z = Z[:1024, :].cuda()
# z = Z[:16, :]
# z = Z[128:256, :].cuda()
z.shape, z.requires_grad

(torch.Size([1024, 3]), True)

In [7]:
# I_hat_onehot, I_hat = VImodule(z)
# I_hat.shape

I_hat = VImodule(z=z[:,:2], labels=z[:,2])
I_hat.shape

torch.Size([100, 100, 10])

In [8]:
I_hat.requires_grad, z.requires_grad

(True, True)

In [11]:
# first 16 datapoints: 5, 0, 4
torch.argmax(I_hat[54][45]), torch.argmax(I_hat[83][52]), torch.argmax(I_hat[67][86])

(tensor(5, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(4, device='cuda:0'))

In [7]:
# first 16 datapoints: 5, 0, 4
torch.argmax(I_hat[540][457]), torch.argmax(I_hat[831][524]), torch.argmax(I_hat[671][866])

(tensor(5, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(4, device='cuda:0'))

In [14]:
I_hat1 = torch.zeros((100,100))

for i in range(100):
    for j in range(100):
        I_hat1[i][j] = torch.argmax(I_hat[i][j])

In [15]:
draw_Ihat(I_hat=I_hat1.detach().cpu())

In [8]:
external_grad = torch.ones(I_hat.shape, dtype=float).cuda()
I_hat.backward(gradient=external_grad)

In [9]:
torch.sum(Z.grad)

tensor(213.0450, dtype=torch.float64)

In [10]:
Z.grad[:16, :]

tensor([[ 234.1367,    0.5643,    0.0000],
        [ 184.6781,    6.6765,    0.0000],
        [   7.7590,  214.8650,    0.0000],
        [  81.8656, -110.8391,    0.0000],
        [  44.3037,  -77.7883,    0.0000],
        [ -91.8296,   31.5204,    0.0000],
        [  37.9358,   76.8595,    0.0000],
        [ -47.8311,  -26.7553,    0.0000],
        [-199.3432,   17.5768,    0.0000],
        [  38.7834, -132.9752,    0.0000],
        [ -31.1355, -162.3155,    0.0000],
        [   2.9657,  223.3813,    0.0000],
        [ 188.2117,   23.6953,    0.0000],
        [-128.5100,  -49.4858,    0.0000],
        [  48.6961,   13.2325,    0.0000],
        [-208.9322,    3.0784,    0.0000]], dtype=torch.float64)